In [119]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader, dataset
import torchvision.transforms as T

In [ ]:
mnist_data_train = datasets.MNIST(root='./data', train=True, download=True, transform=T.ToTensor)
mnist_data_test = datasets.MNIST(root='./data', train=False, download=True, transform=T.ToTensor)

100.0%
100.0%
100.0%
100.0%


In [ ]:
train_dataloader = DataLoader(mnist_data_train, batch_size=64, pin_memory=4, shuffle=True, num_workers=2)
train_dataloader = DataLoader(mnist_data_test, batch_size=64, pin_memory=4, shuffle=True, num_workers=2)

c:\Users\Marwa\Desktop\folders and zips\ai IIM\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [113]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(28 * 28, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 10)
        self.self_activation = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.self_activation(x)
        x = self.layer2(x)
        x = self.self_activation(x)
        x = self.layer3(x)
        x = self.self_activation(x)
        return x

In [114]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
device

'cpu'

In [115]:
model = NeuralNetwork().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [116]:
def train(train_loader:DataLoader,model:nn.Module,  criterion: nn.Module, optimizer: torch.optim):
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [117]:
train(train_dataloader, model, criterion, optimizer)